In [2]:
import os
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
import s3fs
import time
from zoneinfo import ZoneInfo
from datetime import timedelta, datetime


# Set up environments of LakeFS
lakefs_endpoint = os.getenv("LAKEFS_ENDPOINT", "http://lakefs-dev:8000")
ACCESS_KEY = 'access_key'
SECRET_KEY = 'secret_key'

# Setting S3FileSystem for access LakeFS
fs = s3fs.S3FileSystem(
    key=ACCESS_KEY,
    secret=SECRET_KEY,
    client_kwargs={'endpoint_url': lakefs_endpoint}
)

def load_data():
    lakefs_path = "s3://dsi321-record-air-quality/main/airquality.parquet/year=2025"
    data_list = fs.glob(f"{lakefs_path}/*/*/*/*")
    df_all = pd.concat([pd.read_parquet(f"s3://{path}", filesystem=fs) for path in data_list], ignore_index=True)
    df_all['lat'] = pd.to_numeric(df_all['lat'], errors='coerce')
    df_all['long'] = pd.to_numeric(df_all['long'], errors='coerce')
    df_all['year'] = df_all['year'].astype(int)
    df_all['month'] = df_all['month'].astype(int)
    df_all.drop_duplicates(inplace=True)
    df_all['PM25.value'] = df_all['PM25.value'].mask(df_all['PM25.value'] < 0, pd.NA)
    # Fill value "Previous Record" Group By stationID
    df_all['PM25.value'] = df_all.groupby('stationID')['PM25.value'].transform(lambda x: x.ffill())
    return df_all

In [4]:
df = load_data()

In [ ]:
# เปลี่ยน data types กันด้วยนะ

In [ ]:
df.to_csv('data.csv', encoding='utf-8', index=False)

In [ ]:
df.to_parquet('data2.parquet', partition_cols=['year','month','day','hour'], engine='pyarrow')